In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/lib/kaggle/gcp.py
/kaggle/input/checkpoint2/mario_net_4.chkpt


In [3]:
!pip install gym-super-mario-bros

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.8 MB/s eta 0:00:00
  Created wheel for nes-py: filename=nes_py-8.2.1-cp310-cp310-linux_x86_64.whl size=48172 sha256=b438f1e935e1d4dfba4cadfd15635e3d096e8848ef90912b34fe501bcdc1cfca
  Stored in directory: /root/.cache/pip/wheels/34/a7/d5/9aa14b15df740a53d41f702e4c795731b6c4da7925deb8476c
Successfully built nes-py


In [2]:
!pip install git+https://github.com/pytorch-labs/tensordict
!pip install git+https://github.com/pytorch/rl.git

  Cloning https://github.com/pytorch-labs/tensordict to /tmp/pip-req-build-ub3841fm
  Running command git clone --filter=blob:none --quiet https://github.com/pytorch-labs/tensordict /tmp/pip-req-build-ub3841fm
  Resolved https://github.com/pytorch-labs/tensordict to commit 4a34b51f247d7c358602076b676667429950cdcd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tensordict: filename=tensordict-0.4.0+4a34b51-cp310-cp310-linux_x86_64.whl size=299108 sha256=9b762e8fdbde6dc023e00d7c7831b5f3e527425007bb4aa8ac75abaf9fe6a26b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ryhk4j92/wheels/3e/51/3b/af742bd6c05b4e3a7fde597889cfcb9399bfd29535fff78f7b
Successfully built tensordict
  Cloning https://github.com/pytorch/rl.git to /tmp/pip-req-build-xhggv72z
  Running command git clone --filter=blob:none --quiet https://github.com/pytorch/rl.git /tmp/pip-req-build-xhggv72z
  Resolved https://gith

In [20]:
!pip uninstall pyglet -y
!pip install pyglet==1.5.21

^C
Traceback (most recent call last):
  File "/opt/conda/bin/pip", line 6, in <module>
    from pip._internal.cli.main import main
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/ut

In [4]:
import datetime
from pathlib import Path

import gym
import gym_super_mario_bros
import torch
from gym.wrappers import FrameStack
from nes_py.wrappers import JoypadSpace

from torch import nn
import numpy as np
from gym.spaces import Box
from torchvision import transforms as T

from tensordict import TensorDict
from torchrl.data import TensorDictReplayBuffer, LazyMemmapStorage

/opt/conda/lib/python3.10/site-packages/torchrl/data/replay_buffers/samplers.py:37: UserWarning: Failed to import torchrl C++ binaries. Some modules (eg, prioritized replay buffers) may not work with your installation. If you installed TorchRL from PyPI, please report the bug on TorchRL github. If you installed TorchRL locally and/or in development mode, check that you have all the required compiling packages.
  warnings.warn(EXTENSION_WARNING)


In [7]:
class MarioNet(nn.Module):
    """mini CNN structure
  input -> (conv2d + relu) x 3 -> flatten -> (dense + relu) x 2 -> output
  """

    def __init__(self, input_dim, output_dim):
        super().__init__()
        c, h, w = input_dim

        if h != 84:
            raise ValueError(f"Expecting input height: 84, got: {h}")
        if w != 84:
            raise ValueError(f"Expecting input width: 84, got: {w}")

        self.online = self.__build_cnn(c, output_dim)

        self.target = self.__build_cnn(c, output_dim)
        self.target.load_state_dict(self.online.state_dict())

        # Q_target parameters are frozen.
        for p in self.target.parameters():
            p.requires_grad = False

    def forward(self, input, model):
        if model == "online":
            return self.online(input)
        elif model == "target":
            return self.target(input)

    def __build_cnn(self, c, output_dim):
        return nn.Sequential(
            nn.Conv2d(in_channels=c, out_channels=32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )
        
    def td_estimate(self, state, action):
        current_Q = self.net(state, model="online")[
            np.arange(0, self.batch_size), action
        ]  # Q_online(s,a)
        return current_Q

    @torch.no_grad()
    def td_target(self, reward, next_state, done):
        next_state_Q = self.net(next_state, model="online")
        best_action = torch.argmax(next_state_Q, axis=1)
        next_Q = self.net(next_state, model="target")[
            np.arange(0, self.batch_size), best_action
        ]
        return (reward + (1 - done.float()) * self.gamma * next_Q).float()

class Mario:
    def __init__(self, state_dim, action_dim, save_dir, checkpoint = None):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.save_dir = save_dir

        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        # Mario's DNN to predict the most optimal action - we implement this in the Learn section
        self.net = MarioNet(self.state_dim, self.action_dim).float()
        self.net = self.net.to(device = self.device)

        self.exploration_rate = 1
        self.exploration_rate_decay = 0.99999975
        self.exploration_rate_min = 0.1
        self.curr_step = 0

        self.save_every = 1e4  # no. of experiences between saving Mario Net

        self.memory = TensorDictReplayBuffer(storage = LazyMemmapStorage(50000, device = torch.device("cpu")))
        self.batch_size = 32

        self.gamma = 0.9

        self.use_cuda = torch.cuda.is_available()
        if checkpoint:
            self.load(checkpoint)

        self.optimizer = torch.optim.Adam(self.net.parameters(), lr = 0.00025)
        # self.loss_fn = torch.nn.SmoothL1Loss()
        self.loss_fn = torch.nn.MSELoss()

        self.burnin = 1e4  # min. experiences before training
        self.learn_every = 3  # no. of experiences between updates to Q_online
        self.sync_every = 1e4  # no. of experiences between Q_target & Q_online sync

    def act(self, state):
        """
    Given a state, choose an epsilon-greedy action and update value of step.

    Inputs:
    state(``LazyFrame``): A single observation of the current state, dimension is (state_dim)
    Outputs:
    ``action_idx`` (``int``): An integer representing which action Mario will perform
    """
        # EXPLORE
        if np.random.rand() < self.exploration_rate:
            action_idx = np.random.randint(self.action_dim)

        # EXPLOIT
        else:
            state = state[0].__array__() if isinstance(state, tuple) else state.__array__()
            state = torch.tensor(state, device = self.device).unsqueeze(0)
            action_values = self.net(state, model = "online")
            action_idx = torch.argmax(action_values, axis = 1).item()

        # decrease exploration_rate
        self.exploration_rate *= self.exploration_rate_decay
        self.exploration_rate = max(self.exploration_rate_min, self.exploration_rate)

        # increment step
        self.curr_step += 1
        return action_idx

    def cache(self, state, next_state, action, reward, done):
        """
        Store the experience to self.memory (replay buffer)

        Inputs:
        state (``LazyFrame``),
        next_state (``LazyFrame``),
        action (``int``),
        reward (``float``),
        done(``bool``))
        """

        def first_if_tuple(x):
            return x[0] if isinstance(x, tuple) else x

        state = first_if_tuple(state).__array__()
        next_state = first_if_tuple(next_state).__array__()

        state = torch.tensor(state)
        next_state = torch.tensor(next_state)
        action = torch.tensor([action])
        reward = torch.tensor([reward])
        done = torch.tensor([done])

        # self.memory.append((state, next_state, action, reward, done,))
        self.memory.add(TensorDict({ "state": state, "next_state": next_state, "action": action, "reward": reward, "done": done }, batch_size = []))

    def recall(self):
        """
        Retrieve a batch of experiences from memory
        """
        batch = self.memory.sample(self.batch_size).to(self.device)
        state, next_state, action, reward, done = (batch.get(key) for key in ("state", "next_state", "action", "reward", "done"))
        return state, next_state, action.squeeze(), reward.squeeze(), done.squeeze()

    def update_Q_online(self, td_estimate, td_target):
        loss = self.loss_fn(td_estimate, td_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def sync_Q_target(self):
        self.net.target.load_state_dict(self.net.online.state_dict())

    def save(self):
        save_path = (
                self.save_dir / f"mario_net_{int(self.curr_step // self.save_every)}.chkpt"
        )
        torch.save(
            dict(model = self.net.state_dict(), exploration_rate = self.exploration_rate),
            save_path,
        )
        print(f"MarioNet saved to {save_path} at step {self.curr_step}")

    def learn(self):
        if self.curr_step % self.sync_every == 0:
            self.sync_Q_target()

        if self.curr_step % self.save_every == 0:
            self.save()

        if self.curr_step < self.burnin:
            return None, None

        if self.curr_step % self.learn_every != 0:
            return None, None

        # Sample from memory
        state, next_state, action, reward, done = self.recall()

        # Get TD Estimate
        td_est = self.td_estimate(state, action)

        # Get TD Target
        td_tgt = self.td_target(reward, next_state, done)

        # Backpropagate loss through Q_online
        loss = self.update_Q_online(td_est, td_tgt)

        return (td_est.mean().item(), loss)

    def td_estimate(self, state, action):
        current_Q = self.net(state, model = "online")[
            np.arange(0, self.batch_size), action
        ]  # Q_online(s,a)
        return current_Q

    @torch.no_grad()
    def td_target(self, reward, next_state, done):
        next_state_Q = self.net(next_state, model = "online")
        best_action = torch.argmax(next_state_Q, axis = 1)
        next_Q = self.net(next_state, model = "target")[
            np.arange(0, self.batch_size), best_action
        ]
        return (reward + (1 - done.float()) * self.gamma * next_Q).float()

    def load(self, load_path):
        if not load_path.exists():
            raise ValueError(f"{load_path} does not exist")

        ckp = torch.load(load_path, map_location = ('cuda' if self.use_cuda else 'cpu'))
        exploration_rate = ckp.get('exploration_rate')
        state_dict = ckp.get('model')

        print(f"Loading model at {load_path} with exploration rate {exploration_rate}")
        self.net.load_state_dict(state_dict)
        self.exploration_rate = exploration_rate


class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        for i in range(self._skip):
            # Accumulate reward and repeat the same action
            obs, reward, done, trunk, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, trunk, info


class GrayScaleObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape[:2]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def permute_orientation(self, observation):
        # permute [H, W, C] array to [C, H, W] tensor
        observation = np.transpose(observation, (2, 0, 1))
        observation = torch.tensor(observation.copy(), dtype=torch.float)
        return observation

    def observation(self, observation):
        observation = self.permute_orientation(observation)
        transform = T.Grayscale()
        observation = transform(observation)
        return observation


class ResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        if isinstance(shape, int):
            self.shape = (shape, shape)
        else:
            self.shape = tuple(shape)

        obs_shape = self.shape + self.observation_space.shape[2:]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        transforms = T.Compose(
            [T.Resize(self.shape, antialias=True), T.Normalize(0, 255)]
        )
        observation = transforms(observation).squeeze(0)
        return observation


def custom_reward(old, new):
    if old is None:
        return 0

    v = new["x_pos"] - old["x_pos"]
    c = new["time"] - old["time"]
    d = old["life"] - new["life"]
    # p = new["score"] - old["score"]
    reward = 0

    # reward += p / 2

    # if new["status"] == "tall" and old["status"] == "small":
    #    reward += 500

    # if v == 0:
    #    reward -= 10

    reward += v * 3 + c
    
    if d == 1:
        reward = -30

    return reward




def main():
    # print(gym.__version__) # 0.26.2
    # JoypadSpace.reset = lambda self, **kwargs: self.env.reset(**kwargs)
    env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility = True)

    # Apply Wrappers to environment
    env = SkipFrame(env, skip = 4)
    env = GrayScaleObservation(env)
    env = ResizeObservation(env, shape = 84)
    if gym.__version__ < '0.26':
        env = FrameStack(env, num_stack = 4, new_step_api = True)
    else:
        env = FrameStack(env, num_stack = 4)

    # env = JoypadSpace(env, SIMPLE_MOVEMENT)
    # Limit the action-space to
    #   0. walk right
    #   1. jump right
    # env = JoypadSpace(env, [['right'], ['right', 'A'], ['left'], ['A'], ['left', 'A']])
    env = JoypadSpace(env, [['right'], ['right', 'A']])
    # done = True
    # clock = pygame.time.Clock()
    # for step in range(5000):
    #     clock.tick(24)
    #     if done:
    #         state = env.reset()
    #     state, reward, terminated, truncated, info = env.step(env.action_space.sample())
    #     done = terminated or truncated

    #     # state = cv2.cvtColor(state, cv2.COLOR_RGB2GRAY)
    #     # state = cv2.resize(state, (84, 84))

    #     cv2.imshow("State", state)

    #     env.render()

    # env.close() 
    use_cuda = torch.cuda.is_available()
    print(f"Using CUDA: {use_cuda}\n")

    save_dir = Path("checkpoints") / datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
    save_dir.mkdir(parents = True)

    checkpoint = Path("/kaggle/input/checkpoint2/mario_net_4.chkpt")
    # checkpoint = None

    mario = Mario(state_dim = (4, 84, 84), action_dim = env.action_space.n, save_dir = save_dir, checkpoint = checkpoint)

    # logger = MetricLogger(save_dir)
    mario.burnin = 32
    episodes = 5000
    # mario.exploration_rate = 0.4
    for e in range(episodes):
        state = env.reset()

        # Play the game!
        old_info = None
        total_reward = 0
        max_x = 0

        while True:

            # Run agent on the state
            action = mario.act(state)

            # Agent performs action
            next_state, reward, terminated, truncated, info = env.step(action)
            # print(reward)
            # reward = custom_reward(old_info, info)
            # print(reward)
            # old_info = info
            # print(mario.curr_step)
            # print(reward, info)
            done = terminated or truncated
            # next_state, reward, done, trunc, info = env.step(action)
            if mario.curr_step % 5000 == 0:
                print(mario.exploration_rate)

            # Remember
            mario.cache(state, next_state, action, reward, done)

            # Learn
            q, loss = mario.learn()

            # Logging
            # logger.log_step(reward, loss, q)
            # print(reward, loss, q)

            # Update state
            state = next_state
            
            total_reward += reward
            max_x = max(max_x, info["x_pos"])

            # Check if end of game
            if done or info["flag_get"]:
                if info["flag_get"]:
                    print("HA FINITO")
                break
        
        print(f"Episode {e + 1} Total reward: {total_reward}, Max x: {max_x}")
        # logger.log_episode()

        # if (e % 20 == 0) or (e == episodes - 1):
        #     logger.record(episode=e, epsilon=mario.exploration_rate, step=mario.curr_step)



In [ ]:

if __name__ == '__main__':
    main()


Using CUDA: True

Loading model at /kaggle/input/checkpoint2/mario_net_4.chkpt with exploration rate 0.3901239635908362


/opt/conda/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 1 Total reward: 2280.0, Max x: 1138
Episode 2 Total reward: 1252.0, Max x: 899
Episode 3 Total reward: 1473.0, Max x: 1135
Episode 4 Total reward: 2253.0, Max x: 1419
Episode 5 Total reward: 2652.0, Max x: 1138
Episode 6 Total reward: 2245.0, Max x: 1135
Episode 7 Total reward: 1530.0, Max x: 1497
Episode 8 Total reward: 2931.0, Max x: 1406
Episode 9 Total reward: 1740.0, Max x: 898
Episode 10 Total reward: 1859.0, Max x: 1133
0.38963661323279
Episode 11 Total reward: 3914.0, Max x: 2758
Episode 12 Total reward: 2690.0, Max x: 1139
Episode 13 Total reward: 1802.0, Max x: 898
Episode 14 Total reward: 2922.0, Max x: 1410
Episode 15 Total reward: 1421.0, Max x: 1415
Episode 16 Total reward: 1431.0, Max x: 1413
Episode 17 Total reward: 2857.0, Max x: 1137
Episode 18 Total reward: 2002.0, Max x: 877
Episode 19 Total reward: 2261.0, Max x: 1125
Episode 20 Total reward: 671.0, Max x: 296
Episode 21 Total reward: 1684.0, Max x: 1502
0.3891498716821836
MarioNet saved to checkpoints/2024

In [1]:
!zip -r file.zip /kaggle/working/checkpoints/2024-05-22T18-22-07/

	zip warning: name not matched: /kaggle/working/checkpoints/2024-05-22T18-22-07/

zip error: Nothing to do! (try: zip -r file.zip . -i /kaggle/working/checkpoints/2024-05-22T18-22-07/)


In [7]:
from IPython.display import FileLink
FileLink(r'file.zip')

/kaggle/working/file.zip

In [9]:
!rm -r /kaggle/working/checkpoints/2024-05-22T09-01-45/